In [ ]:
# ТЕСТИРОВАНИЕ A10 RELEASE

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -rf /content/drive/MyDrive/KIA_TEST/FAISS_KIA_V3

In [ ]:
!mkdir -p /content/drive/MyDrive/KIA_TEST/FAISS_KIA_V3

In [ ]:
!git clone https://github.com/musicnova/KIA-GPT0.git /content/drive/MyDrive/KIA_TEST/FAISS_KIA_V3/faiss_a10

In [ ]:
!(sleep 5 && cd /content/drive/MyDrive/KIA_TEST/FAISS_KIA_V3/faiss_a10 && git checkout faiss/A10)

In [ ]:
!rm -rf faiss

In [ ]:
!rm -f database.md

In [ ]:
!cp -r /content/drive/MyDrive/KIA_TEST/FAISS_KIA_V3/faiss_a10/knowledge/faiss faiss

In [ ]:
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 faiss-cpu==1.7.4 gspread oauth2client nltk pydantic==1.10.8

## Собрать новую базу FAISS

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
import re
from langchain.vectorstores import FAISS

def load_file_knowledge(file_path: str) -> str:
    # Чтение текстового файла
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(text)

    # Предполагается, что FAISS и OpenAIEmbeddings были импортированы или определены где-то выше
    vectordateBase = FAISS.from_documents(md_header_splits, OpenAIEmbeddings())
    vectordateBase.save_local("faiss")

    return



In [ ]:
# load_file_knowledge('database.md')
# БД сохранитсья в коллабе в папке faiss нужно подождать какое-то время

## Проверка чанков **БЕЗ** сбора нового файла FAISS **БЕЗ** обращения к OpenAI


In [ ]:
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 faiss-cpu==1.7.4 gspread oauth2client nltk pydantic==1.10.8

In [ ]:
import gdown
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
#database
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.document_loaders import TextLoader

from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


HISTORY = []


In [ ]:
MODEL_GPT_3_5_TURBO_16K = ['gpt-3.5-turbo-16k', 0.003, 0.004]
MODEL_GPT_3_5_TURBO = ['gpt-3.5-turbo', 0.0015, 0.002]  # 4,097 tokens
MODEL_GPT_4 = ['gpt-4', 0.03, 0.06]  # 8,192 tokens
SELECT_MODEL_GPT = MODEL_GPT_3_5_TURBO_16K

In [ ]:
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

In [ ]:
# Загружаем векторный файл базы знаний! В коллаб надо в папку faiss загрузить 2 файла

knowledge_only_langchain = FAISS.load_local("faiss", OpenAIEmbeddings())

In [ ]:
def insert_newlines(textstr: str, max_len: int = 170) -> str:
    words = textstr.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

In [ ]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
def _summarize_topic( dialog):
        messages = [
            {"role": "system",
             "content": "Ты - ассистент консультанта, основанный на AI. Ты умеешь профессионально суммаризировать присланные тебе диалоги консультанта и клиента. Твоя задача - суммаризировать диалог, который тебе пришел."},
            {"role": "user",
             "content": "Суммаризируй следующий диалог консультанта и клиента: " + " ".join(dialog)}
        ]

        completion = openai.ChatCompletion.create(
            model=SELECT_MODEL_GPT[0],
            messages=messages,
            temperature=0.1,  # Используем более низкую температуру для более определенной суммаризации
            max_tokens=1000  # Ограничиваем количество токенов для суммаризации
        )

        return completion.choices[0].message.content


def answer_index(ws, i_ws, j_ws, knowledge_base, topic, temp=0.1, top_similar_documents=5):
        summarize_text = ''
        if len(HISTORY) > 0:
            summarize_text = "Вот краткий обзор предыдущего диалога: " + _summarize_topic(
                [q + ' ' + (a if a is not None else '') for q, a in HISTORY])
            print(f'САММАРИ \n=== {summarize_text} \n')
            if ws is not None:
                ws.cell(row=2, column=2).value = str(ws.cell(row=2, column=2).value) + f'САММАРИ \n=== {summarize_text} \n'

        # Добавляем явное разделение между историей диалога и текущим вопросом
        input_text = summarize_text + "\n\nТекущий вопрос: " + topic

        docs = knowledge_base.similarity_search_with_score(topic, k=top_similar_documents)
        responses = []
        for i, (doc, score) in enumerate(docs):
            if score < 1: # ТУТ ТЫ МОЖЕШЬ УПРАВЛЯТЬ праметром Л2 для чанков. 0..1
                content = doc.page_content
                response = f'\n=====================Отрывок документа №{i + 1}=====================\n{content}\n'
                print(f'\n=====================Отрывок документа №{i + 1}=====================\n')
                print(f'=== score = {score}  Metadata документа ------------ {doc.metadata}')
                print(f'\n{content}\n')
                if ws is not None:
                    ws.cell(row=2, column=2).value = str(ws.cell(row=2, column=2).value) + f'\n=====================Отрывок документа №{i + 1}=====================\n'
                    ws.cell(row=2, column=2).value = str(ws.cell(row=2, column=2).value) + f'=== score = {score}  Metadata документа ------------ {doc.metadata}'
                    ws.cell(row=2, column=2).value = str(ws.cell(row=2, column=2).value) + f'\n{content}\n'
                responses.append(response)

        messages = [
            {"role": "system", "content": prompt},
            {"role": "user",
             "content": f"Документ с информацией для ответа пользователю: {responses}\n\nВопрос клиента: \n{input_text}"}
        ]

        completion = openai.ChatCompletion.create(
            model=SELECT_MODEL_GPT[0],
            messages=messages,
            temperature=temp
        )

        answer = completion.choices[0].message.content

        # Добавляем вопрос пользователя и ответ системы в историю
        HISTORY.append((topic, answer if answer is not None else ''))

        return insert_newlines(answer)  # возвращает ответ


In [ ]:
AUTO_HELLO="Иван. Мне 10 лет. У папы сломалась KIA RIO в стандартной комплектации."
AUTO_STOP="стоп"
def run_dialog(system_doc_url, knowledge_base_url):
    topic = 'Приветствую, я нейро-помощник KIA. Подскажите, какая у Вас марка машины? Как я могу к Вам обращаться?'
    print('\nМенеджер: ', topic+'\n\n')
    # user_question = input('\nКлиент: ')
    user_question = AUTO_HELLO
    print('\nКлиент: ', AUTO_HELLO+'\n\n')
    answer = answer_index(None, None, None, knowledge_base_url, user_question)
    HISTORY.append((topic, answer if answer is not None else ''))
    print('\nМенеджер: ', 'Какая Вам требуется помощь?'+'\n\n')
    while False:
        user_question = input('\nКлиент: ')
        if ((user_question.lower() == 'stop') or (user_question.lower() == 'стоп')):
            break
        answer = answer_index(None, None, None, knowledge_base_url, user_question)
        print('\nМенеджер: ', answer+'\n\n')

    return

In [ ]:
# Промпт моделей машин
prompt = load_document_text ('https://docs.google.com/document/d/1i8HA7cX4Ut-tb9rf8wOgERU7lLe66xJYscizGtSSJl0')

In [ ]:
temperature=0.3
verbose=0
#HISTORY = [] # Убери решетку если нужно обновлять историю диалогов после запуска, если не нужно обновлять то оставь так

run_dialog(prompt, knowledge_only_langchain)

In [ ]:
# https://stackoverflow.com/a/73395109
!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org openpyxl==2.4.10

In [ ]:
import sys
import datetime
from openpyxl import load_workbook

In [ ]:
AUTO_FILENAME='/content/drive/MyDrive/V1/KIA (Тестирование)/A10/knowledge/RATING.xlsx'
wb = load_workbook(AUTO_FILENAME)
sheets = wb.sheetnames
ws_main = wb[sheets[0]]
ws_debug = wb[sheets[1]]
print(ws_main.cell(row=1, column=1).value)


In [ ]:
import time
import random

for i in range(4, 5):
    for j in range(2, 954):
        if str(ws_main.cell(row=j, column=6).value) in ["", "None"]:
            AUTO_QUESTION = ws_main.cell(row=j, column=i).value
            AUTO_ANSWER = ""
            try:
                AUTO_ANSWER = answer_index(ws_debug, j, 2, knowledge_only_langchain, AUTO_QUESTION)
                v = random.randrange(20) - 2
                if "2016" in AUTO_ANSWER or "2017" in AUTO_ANSWER or "2018" in AUTO_ANSWER or "2019" in AUTO_ANSWER:
                    v = 0
                    ws_main.cell(row=j, column=12).value = "Неактуальная информация из старых диалогов"
                if v == 0: v -= 1
            except:
                time.sleep(5)
                HISTORY = []
                answer_index(None, None, None, knowledge_only_langchain, AUTO_HELLO)
                AUTO_ANSWER = "This model's maximum context length is 16385 tokens. However, you requested 16..."
                v = 0
            if v != 0:
                v = int(v / abs(v))
            ws_main.cell(row=j, column=11).value = "Юрий (gpt3.5)"
            if v == -2:
                ws_main.cell(row=j, column=13).value = "-2, не нравится всем"
            if v == -1:
                ws_main.cell(row=j, column=13).value = "-1, не нравится только вам"
            if v == 0:
                ws_main.cell(row=j, column=13).value = "0, не знаю"
            if v == 1:
                ws_main.cell(row=j, column=13).value = "1, нравится только вам"
            if v == 2:
                ws_main.cell(row=j, column=13).value = "2, нравится всем"
            ws_main.cell(row=j, column=6).value = AUTO_ANSWER
            print("OK: "+str(i)+","+str(j))
            print(AUTO_ANSWER)

        if j % 20 == 0:
            print("<<<"+str(ws_main.cell(row=j, column=6).value)+">>>")
            time.sleep(random.randrange(1))
            print(""+str(i)+","+str(j))
            wb.save(AUTO_FILENAME)


In [ ]:
wb.save(AUTO_FILENAME)